In [24]:
from google.colab import drive
drive.mount('/content/gdrive/')
import os
root = "/content/gdrive/My Drive/Colab Notebooks/a/vit"
# if not os.path.exists(root):
#     os.makedirs(root)
os.chdir(root)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [25]:
pip install datasets

In [26]:
# import os
# import gdown


# # import urllib.request
# dir_data = 'data'
# output_splits_dir = os.path.join(dir_data, 'official_splits')
# if not os.path.isdir(output_splits_dir):
#     # self.dir_data = dir_data
#     # Download the file using urllib
#     filename = 'nyu_depth_v2_labeled.mat'
#     os.makedirs(dir_data,exist_ok=True)
#     # urllib.request.urlretrieve(url, os.path.join(dir_data,filename))
#     file_path = os.path.join(dir_data,filename)
#     if not os.path.isfile(file_path):
#         url = 'http://horatio.cs.nyu.edu/mit/silberman/nyu_depth_v2/nyu_depth_v2_labeled.mat'
#         gdown.download(url, output = file_path, quiet=False)
#     else:
#         print('mat exist')
# # Extract official train-test splits using custom script
# from extract_official_train_test_set_from_mat import extract_official_splits

# # Provide the necessary arguments and run the extraction script
# input_mat_file = os.path.join(dir_data, 'nyu_depth_v2_labeled.mat')
# splits_file = os.path.join(dir_data,'splits.mat')
# extract_official_splits(input_mat_file, splits_file, output_splits_dir)


In [27]:
import os
import sys
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from tqdm import tqdm

import save as SAVE
import data as DATA
import models as MODELS

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device


'cuda:0'

In [28]:
device = 'cpu'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [29]:
TEST = False
OUTPUT_ROOT = 'output'
DATA_DIR = 'data'

In [54]:
BACKBONE_TYPE = 'resnet' #'mae','resnet'
TASK = 'depth' #'classification', 'segmentation', 'depth'
MODE = 'e2e' #'last','e2e'
DATA_NAME = 'nyudepthv2' #cifer10,nyudepthv2
ltr = 4000
lt = 200
SAVE.init(OUTPUT_ROOT,BACKBONE_TYPE,TASK,MODE,DATA_NAME,TEST,new='onl1')
DATA.init(DATA_DIR)

# normalize_data = {
#     'resnet':True,
#     'mae':False
# }[BACKBONE_TYPE]
# preprocessor = MODELS.get_preprocessor(BACKBONE_TYPE)

In [55]:
N_WORKERS = 24 #0 if parallelism fails
BS = 16
EPOCH = 30
LR = 1e-4

if TEST:
    N_WORKERS = 4 #0 if parallelism fails
    BS = 2
    EPOCH = 20
    head_n = 15

# Model

In [56]:
def count_model_size(model):
    size = 0
    for name, param in model.named_parameters():
        size += param.numel()
    return size

In [57]:
import importlib
importlib.reload(MODELS)
# import models as MODELS

<module 'models' from '/content/gdrive/.shortcut-targets-by-id/1ntiOq49ghE2mFRVQi4zj2bgcOgWXfUf9/Colab Notebooks/a/vit/models.py'>

In [58]:
model, train = MODELS.get_model(BACKBONE_TYPE,TASK,MODE)
# preprocessor = MODELS.get_preprocessor(BACKBONE_TYPE)
model = model.to(device)
count_model_size(model)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


13681520

# Dataset

In [59]:
import importlib
importlib.reload(DATA)
# import data as DATA

<module 'data' from '/content/gdrive/.shortcut-targets-by-id/1ntiOq49ghE2mFRVQi4zj2bgcOgWXfUf9/Colab Notebooks/a/vit/data.py'>

In [60]:
trainset ,ltrain= DATA.get_set(DATA_NAME,cache_dir=DATA_DIR, train=True, download=True)
testset  ,lval= DATA.get_set(DATA_NAME,cache_dir=DATA_DIR, train=False, download=True)
ltrain = ltrain//BS
lval = lval//BS
# testset = DATA.head(testset,head_n)
# if TEST:
#     trainset = DATA.head(trainset,head_n)
#     testset = DATA.head(testset,head_n)

from torch.utils.data import SubsetRandomSampler

# Define the indices for the subset of 2000 samples
sampler = SubsetRandomSampler(list(range(len(trainset))))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BS,
                                          shuffle=False, num_workers=N_WORKERS,sampler = sampler)


testloader = torch.utils.data.DataLoader(testset, batch_size=BS,
                                         shuffle=False, num_workers=N_WORKERS)



Loading dataset shards:   0%|          | 0/48 [00:00<?, ?it/s]

In [61]:
len(trainloader)

2974

# Train

In [62]:
class MaskedL1Loss(nn.Module):
    def __init__(self):
        super(MaskedL1Loss, self).__init__()

    def forward(self, pred, target):
        assert pred.dim() == target.dim(), "inconsistent dimensions"
        valid_mask = (target > 0).detach()
        diff = target - pred
        diff = diff[valid_mask]
        self.loss = diff.abs().mean()
        return self.loss
class berHuLoss(nn.Module):
    def __init__(self):
        super(berHuLoss, self).__init__()

    def forward(self, pred, target):
        assert pred.dim() == target.dim(), "inconsistent dimensions"

        huber_c = torch.max(pred - target)
        huber_c = 0.2 * huber_c

        valid_mask = (target > 0).detach()
        diff = target - pred
        diff = diff[valid_mask]
        diff = diff.abs()

        huber_mask = (diff > huber_c).detach()

        diff2 = diff[huber_mask]
        diff2 = diff2 ** 2

        self.loss = torch.cat((diff, diff2)).mean()

        return self.loss


# criterion = MaskedL1Loss()
criterion = berHuLoss()
if MODE == 'last':
    params = model.added.parameters()
if MODE == 'e2e':
    params = model.parameters()

optimizer = optim.AdamW(params, lr=LR)
num_epochs = EPOCH
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.8)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=3)

In [63]:
from metrics import Result, AverageMeter

def evaluate_model(loader):

    eval_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(loader, 0), total=lval):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data.values() if isinstance(data, dict) else data
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                pred = train(model,inputs)
            loss = criterion(pred,labels).item()
            eval_loss += loss
            result = Result()
            result.evaluate(pred,labels)
    return eval_loss/lval, result


In [64]:
data = next(iter(trainloader))

In [65]:
inputs,labels=data.values() if isinstance(data, dict) else data

In [66]:
inputs = inputs.to(device)
labels = labels.to(device)

In [67]:
inputs.shape

torch.Size([16, 3, 224, 224])

In [68]:
y=train(model,inputs)

In [69]:
y.shape

torch.Size([16, 1, 224, 224])

In [70]:
# train_losses = []
# test_losses = []
# test_acc = []

test_loss_min = np.Inf
min_rmse = np.Inf
train_losses, test_losses, test_acc = list(map(list,SAVE.load_log()))

In [71]:

# Train
eval_freq = 1
for epoch in range(num_epochs):  # loop over the dataset multiple times
    model.train()
    train_loss_buffer = []
    for i, data in zip(tqdm(range(ltrain//BS)),trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data.values() if isinstance(data, dict) else data
        # inputs = preprocessor(inputs)
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # inputs = preprocessor(inputs, return_tensors="pt").to(device)

        # outputs = model(**inputs).logits

        # loss = criterion(outputs, labels)
        pred = train(model,inputs)
        loss = criterion(pred,labels)
        train_loss_buffer.append(loss.item())
        loss.backward()
        optimizer.step()

    mean_train_loss = np.mean(train_loss_buffer)
    print(f'[{epoch}] train loss: {mean_train_loss}')
    train_losses.append(mean_train_loss)
    train_loss_buffer.clear()

    # Update evaluation on test set
    if epoch % eval_freq == 0:
        print('update evaluation')
        model.eval()
        test_loss, evaluation = evaluate_model(testloader)

    test_losses.append(test_loss)
    test_acc.append(evaluation.mse)

    print(f'''
            [{epoch}]
            train loss: {train_losses[-1]:.3f},
            valid_loss: {test_loss:.3f},
            mse: {evaluation.mse},
            lr={optimizer.param_groups[0]["lr"]}
        ''')
    result = evaluation
    print(
        "rmse={:.3f}, rml={:.3f}, log10={:.3f}, d1={:.3f}, d2={:.3f}, dd31={:.3f}, "
        "t_gpu={:.4f}".
            format(result.rmse, result.absrel, result.lg10, result.delta1, result.delta2,
                    result.delta3,
                    result.gpu_time))

    # save model if validation loss has decreased
    if test_loss < test_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).'.format(
        test_loss_min,
        test_loss))
        test_loss_min = test_loss
    if result.rmse<min_rmse:
        print('rmse decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        min_rmse,
        result.rmse))
        SAVE.save_model(model)
        min_rmse = result.rmse
    # scheduler.step()
    SAVE.save_log(train_losses,test_losses,test_acc)
    scheduler.step(train_losses[-1])
    # scheduler.step(test_acc[-1]

print('Finished Training')

100%|██████████| 185/185 [00:31<00:00,  5.89it/s]


[0] train loss: 5.192327953673698
update evaluation


13it [00:01,  7.66it/s]



            [0]
            train loss: 5.192,
            valid_loss: 2.501,
            mse: 0.916793942451477,
            lr=0.0001
        
rmse=0.957, rml=0.401, log10=nan, d1=0.498, d2=0.789, dd31=0.872, t_gpu=0.0000
Validation loss decreased (inf --> 2.501367).
rmse decreased (inf --> 0.957494).  Saving model ...


100%|██████████| 185/185 [00:32<00:00,  5.71it/s]


...


            [28]
            train loss: 0.921,
            valid_loss: 1.745,
            mse: 0.7499781847000122,
            lr=0.0001
        
rmse=0.866, rml=0.289, log10=0.109, d1=0.542, d2=0.880, dd31=0.945, t_gpu=0.0000
Validation loss decreased (1.756140 --> 1.744939).


100%|██████████| 185/185 [00:31<00:00,  5.88it/s]


[29] train loss: 0.943185497296823
update evaluation


13it [00:00, 16.38it/s]                       


            [29]
            train loss: 0.943,
            valid_loss: 1.705,
            mse: 0.5227105021476746,
            lr=0.0001
        
rmse=0.723, rml=0.253, log10=0.094, d1=0.627, d2=0.895, dd31=0.969, t_gpu=0.0000
Validation loss decreased (1.744939 --> 1.704990).
Finished Training


In [48]:

min_rmse = 0.l8

In [49]:
SAVE.save_model(model)

In [50]:
train_losses, test_losses, test_acc = list(map(list,SAVE.load_log()))
test_acc

[1.3153940439224243,
 0.6530229449272156,
 0.718403697013855,
 0.5319786071777344,
 0.5023412704467773,
 0.47253724932670593,
 0.686877429485321,
 1.1733492612838745,
 1.2270843982696533,
 1.0555338859558105,
 0.8277010321617126,
 1.094414234161377,
 1.336553692817688,
 1.1186192035675049,
 1.0707905292510986,
 0.9272807836532593,
 1.0285307168960571,
 0.9948365092277527,
 0.9569809436798096,
 0.8770530819892883,
 0.9955605864524841,
 0.9569492340087891,
 0.9343591332435608,
 1.0552898645401,
 0.9387233257293701,
 0.9692915081977844,
 0.9411337375640869,
 0.957649827003479,
 0.984296977519989,
 0.9804324507713318]

In [51]:
model.eval()
_,result=evaluate_model(testloader)

13it [00:03,  3.56it/s]


In [52]:
result.delta1

0.4300462305545807

In [53]:
print(
        """
        |rmse|rel|log10|d1|d2|d3|time in gpu|
        | -- | -- | -- | -- | -- | -- |
        |{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.4f}|
        """.format(
            result.rmse, result.absrel, result.lg10, result.delta1, result.delta2,
                    result.delta3, result.gpu_time)

SyntaxError: incomplete input (<ipython-input-53-18725300f845>, line 8)

In [ ]:
torch.save(model.state_dict(), 'output/model.pt')

In [ ]:
%pwd